# orbitWebApp

## Description:
This notebook provides an application to visualize a single (guiding-centre) orbit and its
path around the tokamak in an interactive and intuitive manner. The (E,pm,Rm) coordinate
is specified and the orbit can be visualize via sliders. 

## Inputs:
- folderpath_OWCF - The path to the OWCF folder on your computer. Needed for correct loading - String
- filepath_equil - The path to the .eqdsk-file (or .jld2-file) with the tokamak magnetic equilibrium and geometry - String
- FI_species - The species of the particle being simulated (deuterium, tritium etc) - String
- anim_numOtau_p - The number of poloidal transit times of the orbit trajectory animation - Int64 (or Float64)
- verbose - If set to true, the app will talk a lot! - Bool

## Outputs:
# -

## Saved files:
# -

### Original script written by Henrik Järleblad, henrikj@dtu.dk
### Animation and GIF-saving tools added by Andrea Valentini, anvalen@dtu.dk
### Last maintained 2025-10-07

In [1]:
# SPECIFY THE INPUTS IN THIS CELL

# Please specify the OWCF folder and let the notebook change directory to the 
# OWCF folder when the cell below is executed. This is to be able to load the
# correct versions of the Julia packages as specified in the Project.toml and 
# Manifest.toml files.
folderpath_OWCF = "/path/to/the/OWCF/" # Finish with '/'

filepath_equil = "path/to/a/magnetic/equilibrium/file.eqdsk" # Or .jld2
FI_species = "D" # Example deuterium: "D"
anim_numOtau_p = 3 # If the button 'include_anim' is toggled to 'on' in the web application, the length of the animation of 
# the orbit trajectory will correspond to this number of poloidal transit times
# Please note: If the 'include_anim' and 'save_plots' buttons are both toggled to 'on' in the 
# interactive web application, a GIF of the orbit trajectory animation will be saved (takes a LONG time).
# Also, please note, the saved gif will use more frames and higher framerate (60s^-1) for better quality
verbose = true
extra_kw_args = Dict(:toa => true, :limit_phi => true)
# toa is 'try only adaptive'
# limits the number of toroidal turns for orbits

Dict{Symbol, Bool} with 2 entries:
  :limit_phi => 1
  :toa       => 1

In [4]:
# RUN THE APP BY FIRST SELECTING THE ABOVE CELL AND PRESS Ctrl+Enter. THEN SELECT THIS CELL
# AND PRESS Shift+Enter. MIGHT TAKE A MOMENT TO PREPARE+LOAD THE INTERACTIVE APP.

cd(folderpath_OWCF)
using Pkg
Pkg.activate(".")

verbose && println("Loading packages... ")
using Interact
using EFIT
using Equilibrium
using GuidingCenterOrbits
using Plots
using JLD2
using FileIO
using Mux
using WebIO
using Suppressor
include(folderpath_OWCF*"misc/species_func.jl")

## ------
verbose && println("Loading magnetic equilibrium... ")
M, wall, jdotb = nothing, nothing, nothing # Initialize global magnetic equilibrium variables
try
    global M; global wall; global jdotb # Declare global scope
    M, wall = read_geqdsk(filepath_equil,clockwise_phi=false) # Assume counter-clockwise phi-direction
    jdotb = M.sigma # The sign of the dot product between the plasma current and the magnetic field
catch # Otherwise, assume magnetic equilibrium is a saved .jld2 file
    global M; global wall; global jdotb; local myfile # Declare global scope and local scope for variables
    myfile = jldopen(filepath_equil,false,false,false,IOStream)
    M = myfile["S"]
    wall = myfile["wall"]
    close(myfile)
    jdotb = (M.sigma_B0)*(M.sigma_Ip)
end

#########################################################################################
verbose && println("Computing flux function on 100x100 (R,z)-grid (to plot flux surfaces)... ")
flux_r = range(extrema(wall.r)...,length=100)
flux_z = range(extrema(wall.z)...,length=100)
inds = CartesianIndices((length(flux_r),length(flux_z)))
psi_rz = [M(flux_r[ind[1]], flux_z[ind[2]]) for ind in inds]
psi_mag, psi_bdry = psi_limits(M)

## ------
# The web application
verbose && println("Building web application (might take approx. 1-2 min)... ")
R_hfs = minimum(wall.r) # R-coord of high-field side wall
R_lfs = maximum(wall.r) # R-coord of low-field side wall
phi = collect(0:1:359).*(2*pi/180.0) # Toroidal angle
topview_R_hfs_x = (R_hfs).*cos.(phi)
topview_R_hfs_y = (R_hfs).*sin.(phi)
topview_R_lfs_x = (R_lfs).*cos.(phi)
topview_R_lfs_y = (R_lfs).*sin.(phi)

@suppress begin # Suppress annoying and unnecessary warnings etc
@manipulate for tokamak_wall = Dict("on" => true, "off" => false), 
    E = 5.0, pm = 0.5, Rm = (maximum(wall.r)+magnetic_axis(M)[1])/2, i=collect(1:1:500), 
    include_anim = Dict("on" => true, "off" => false), save_plots = Dict("on" => true, "off" => false)
    rm("$(folderpath_OWCF)temp.gif",force=true) # Make sure to always clean up any unwanted animation .gif files

    if i==1 # Stupid fix to a stupid bux (A. Valentini, anvalen@dtu.dk)
        i=2
    end

    EPRc = EPRCoordinate(M, E, pm, Rm, amu=getSpeciesAmu(FI_species), q=getSpeciesEcu(FI_species))
    o = get_orbit(M,EPRc; wall=wall, interp_dt=1.0e-10, max_length=500, extra_kw_args...) # interp_dt is set to ridiculously small value, to ensure orbit path length of 500
    
    if include_anim
        gcp = GCParticle(EPRc) # Define the guiding-center (GC) particle object
        # Intergrate. Get the orbit path (path) and the orbit status (stat) objects
        path, stat = integrate(M, gcp; one_transit=false, r_callback=false, wall=wall, interp_dt=1.0e-10, max_length=5000, tmax=anim_numOtau_p*o.tau_p)
        gcvalid = gcde_check(M, gcp, path) # Check if usage of the guiding-center approximation was valid, given the length scale of the variation of the magnetic field

        rmax = stat.rm
        if stat.class != :incomplete && stat.class != :lost # If the orbit was not incomplete, nor lost...
            if rmax > EPRc.r && (false || !isapprox(rmax,EPRc.r,rtol=1e-4)) # If it ended up outside of its initial R coordinate, or it's not exact...
                stat.class = :invalid # It's invalid!
            end
        else
            stat.tau_p=zero(stat.tau_p) # Else, it's an incomplete or lost orbits, and has no poloidal...
            stat.tau_t=zero(stat.tau_t) # ...and toroidal transit times
        end
        o_long = Orbit(EPRc,o.class,stat.tau_p,stat.tau_t,path,gcvalid)

        mv_o_x = cos.(o_long.path.phi).*(o_long.path.r) # Compute x-coordinates for the animation orbit trajectory
        mv_o_y = sin.(o_long.path.phi).*(o_long.path.r) # Compute y-coordinates for the animation orbit trajectory
    end

    topview_o_x = cos.(o.path.phi).*(o.path.r)
    topview_o_y = sin.(o.path.phi).*(o.path.r)

    orb_color = :black
    orb_linestyle = :solid
    if o.class==:invalid
        orb_color = :gray
        orb_linestyle = :dash
    elseif o.class == :lost
        orb_color = :brown
    elseif o.class == :incomplete # If this happens, you are in trouble. Because it will likely take forever to calculate. Please just re-start the app instead.
        orb_color = :yellow
    elseif o.class == :trapped
        orb_color = :blue
    elseif o.class == :co_passing
        orb_color = :green
    elseif (o.class == :stagnation && o.coordinate.r>=magnetic_axis(M)[1]) # Regular stagnation orbit
        orb_color = :red
    elseif o.class == :potato
        orb_color = :orange
    elseif o.class == :ctr_passing
        orb_color = :purple
    elseif (o.class == :stagnation && o.coordinate.r<magnetic_axis(M)[1]) # Counter-stagnation orbit
        orb_color = :pink
    else
        error("Something's gone wrong!!! Orbit class unknown!")
    end

    #topview plot
    plt_top = Plots.plot(topview_o_x[1:i],topview_o_y[1:i],label="$(o.class) orbit", color=orb_color, linestyle=orb_linestyle, linewidth=1.5)
    plt_top = Plots.plot!(topview_R_lfs_x,topview_R_lfs_y, label="JET wall", color=:black, linewidth=1.5)
    plt_top = Plots.plot!(topview_R_hfs_x,topview_R_hfs_y, label="", color=:black,linewidth=1.5, aspect_ratio=:equal, title="Top view")
    plt_top = Plots.scatter!([topview_o_x[i]],[topview_o_y[i]],label="",mc=orb_color, xlabel="x [m]", ylabel="y [m]")
    if save_plots
        png(plt_top, "plt_top_$(round(E, digits=2))_$(round(o.coordinate.pitch, digits=2))_$(round(o.coordinate.r,digits=2))")
    end

    #cross-sectional plot
    plt_crs = Plots.scatter([magnetic_axis(M)[1]],[magnetic_axis(M)[2]],label="Magnetic axis", mc=:gray, aspect_ratio=:equal, xlabel="R [m]", ylabel=" z[m]",title="E: $(round(E,digits=2)) keV  pm: $(round(o.coordinate.pitch, digits=2))  Rm: $(round(o.coordinate.r,digits=2))")
    plt_crs = Plots.plot!(o.path.r[1:i],o.path.z[1:i], label="$(o.class) orbit", color=orb_color, linestyle=orb_linestyle, linewidth=1.5)
    if tokamak_wall
        plt_crs = Plots.contour!(flux_r,flux_z,psi_rz',levels=collect(range(psi_mag,stop=psi_bdry,length=5)),color=:gray, linestyle=:dot,linewidth=1.5, label="",colorbar=false)
        plt_crs = Plots.plot!(wall.r,wall.z, label="JET wall", color=:black, linewidth=1.5)
    end
    plt_crs = Plots.scatter!([o.path.r[i]],[o.path.z[i]],mc=orb_color,label="")
    if save_plots
        png(plt_crs, "plt_crs_$(round(E, digits=2))_$(round(o.coordinate.pitch, digits=2))_$(round(o.coordinate.r,digits=2))")
    end

    #pitch visualization plot
    t_array = collect(range(0.0,stop=((o.tau_p)*(i/length(o.path.pitch))), length=i)) ./(1.0e-6) # Microseconds
    plt_pitc = Plots.plot(t_array, o.path.pitch[1:i],color=orb_color,title="Pitch (p) along orbit path \n p=$(round(o.path.pitch[i],sigdigits=3))",label="", xlabel="Poloidal time [microseconds]")
    plt_pitc = Plots.scatter!([t_array[end]],[o.path.pitch[i]],color=orb_color,label="", ylabel="pitch [-]")
    if save_plots
        png(plt_pitc, "plt_pitc_$(round(E, digits=2))_$(round(o.coordinate.pitch, digits=2))_$(round(o.coordinate.r,digits=2))")
        !include_anim && println("Plots saved successfully. Please switch off 'save_plots'.")
    end

    #top view movie
    if include_anim
        #plt_anim = Plots.plot((mv_o_x)[1:2],(mv_o_y)[1:2],color=orb_color,alpha=0.3)
        #plt_anim = Plots.plot!((mv_o_x)[1:2],(mv_o_y)[1:2],color=orb_color,alpha=0.8)
        #plt_anim = Plots.scatter!([(mv_o_x)[1]],[(mv_o_y)[1]],color=orb_color)
        #plt_anim = Plots.plot!(topview_R_hfs_x, topview_R_hfs_y, color=:black)
        #plt_anim = Plots.plot!(topview_R_lfs_x, topview_R_lfs_y, color=:black)
        # To achieve fast and efficient animation plotting, trick 1 is to not plot the axis ticks...
        #plt_anim = plot!(plt_anim,aspect_ratio=:equal, legend=false, xlabel="x [m]", ylabel="y [m]",xticks=false,yticks=false,left_margin=4Plots.mm)
        # ...and trick 2 is to define the needed indexes outside of the @animate loop
        A,B = -500:1:length(mv_o_x)-500, -50:1:length(mv_o_x)-50 # We want negative indexes in for the first 50, 500, etc ones...
        A,B = ifelse.(A .> 0, A, 1), ifelse.(B .> 0, B, 1) # ...so that we can easily put them to 1 in this next line.
        
        plt_movie = @animate for j=1:50:length(mv_o_x)
            plt_anim = Plots.plot(aspect_ratio=:equal, legend=false, xlabel="x [m]", ylabel="y [m]",xticks=false,yticks=false,left_margin=4Plots.mm)
            plt_anim = Plots.plot!((mv_o_x)[A[j]:j],(mv_o_y)[A[j]:j],color=orb_color,alpha=0.3)
            plt_anim = Plots.plot!((mv_o_x)[B[j]:j],(mv_o_y)[B[j]:j],color=orb_color,alpha=0.8)
            plt_anim = Plots.scatter!([(mv_o_x)[j]],[(mv_o_y)[j]],color=orb_color)
            plt_anim = Plots.plot!(topview_R_hfs_x, topview_R_hfs_y, color=:black)
            plt_anim = Plots.plot!(topview_R_lfs_x, topview_R_lfs_y, color=:black)
            plt_anim
        end every 1 # And create a "movie object" 'plt_movie' from every frame ('every 1')
        if save_plots
            save_movie = @animate for j=1:10:length(mv_o_x)
                plt_anim = Plots.plot(title="Tokamak time=$(round(j*(anim_numOtau_p*o.tau_p*10^6)/5000, digits=0))/$(round(anim_numOtau_p*o.tau_p*10^6, digits=0)) [μs]")
                plt_anim = Plots.plot!(aspect_ratio=:equal, legend=false, xlabel="x [m]", ylabel="y [m]",xticks=false,yticks=false,left_margin=4Plots.mm)
                plt_anim = Plots.plot!((mv_o_x)[A[j]:j],(mv_o_y)[A[j]:j],color=orb_color,alpha=0.3)
                plt_anim = Plots.plot!((mv_o_x)[B[j]:j],(mv_o_y)[B[j]:j],color=orb_color,alpha=0.8)
                plt_anim = Plots.scatter!([(mv_o_x)[j]],[(mv_o_y)[j]],color=orb_color)
                plt_anim = Plots.plot!(topview_R_hfs_x, topview_R_hfs_y, color=:black)
                plt_anim = Plots.plot!(topview_R_lfs_x, topview_R_lfs_y, color=:black)
                plt_anim
            end every 1 # Do the same when saving the movie, but with a higher fps (j=1:10:length(mv_o_x) and fps=60)
            gif(save_movie,"plt_anim_$(round(E, digits=2))_$(round(o.coordinate.pitch, digits=2))_$(round(o.coordinate.r,digits=2)).gif",fps=20)
            println("Movie and plots saved successfully! Please switch off 'save_plots'.")
        end
    end 
    if !include_anim
        vbox(vskip(1em),
            md"Warning messages below the app can be ignored.",
            md"**Please note, when switching the 'include\_anim' button from off to on, it might a couple of seconds for the app to load.**",
            vskip(1em),
            hbox(Plots.plot(plt_top),Plots.plot(plt_crs)),
            hbox(Plots.plot(plt_pitc))
        )
    else
        vbox(vskip(1em),
            md"Warning messages below the app can be ignored.",
            md"**Please note, when switching the 'include\_anim' button from off to on, it might take a couple of seconds for the app to load.**",
            vskip(1em),
            md"*Info print below app regarding saved 'temp.gif' can be ignored. The 'temp.gif' files are automatically deleted.*",
            md"*Including animations reduces the response time of the app by 3-4 seconds. Saving animations can take time (approx. 30 seconds)*",
            md"**Therefore, please set 'include\_anim' to off when not needed, and 'save\_plots' to off when movie save is completed.**",
            md"*Please wait for confirmation in the log below the app for when movie/plots have been saved.*",
            vskip(1em),
            hbox(Plots.plot(plt_top),Plots.plot(plt_crs)),
            hbox(Plots.plot(plt_pitc),gif(plt_movie,"temp.gif",fps=15))
        )
    end
end
end

Loading packages... 
Loading magnetic equilibrium... 


  Activating project at `~/Codes/OWCF`


Computing flux function on 100x100 (R,z)-grid (to plot flux surfaces)... 
Building web application (might take approx. 1-2 min)... 


Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Scope(Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :label), Any["tokamak_wall"], Dict{Symbol, Any}(:className => "interact ", :style => Dict{Any, Any}(:padding => "5px 10px 0px 10px")))], Dict{Symbol, Any}(:className => "interact-flex-row-left")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :span), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :label), Any[], Dict{Symbol, Any}(:attributes => Dict("data-bind" => "text : key")))], Dict{Symbol, Any}(:attributes => Dict("data-bind" => "click: function (){\$root.update(val)}, css: {'is-primary is-selected' : \$root.index() == val, 'interact-widget is-medium button ' : true}")))], Dict{Symbol, Any}(:attributes => ("data-bind" => "foreach : options_js"), :className => "buttons has-addons is-centered"))], Dict{Symbol, Any}(:className => "interact-flex-row-center")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[], Dict{Symbol, Any}())], Dict{Symbol, Any}(:className => "interact-flex-row-right"))], Dict{Symbol, Any}(:className => "interact-flex-row interact-widget")), Dict{String, Tuple{AbstractObservable, Union{Nothing, Bool}}}("options_js" => (Observable(OrderedDict[OrderedDict{String, Any}("key" => "off", "val" => 1, "id" => "idtg3iLrcE"), OrderedDict{String, Any}("key" => "on", "val" => 2, "id" => "idDjtE3fYE")]), nothing), "index" => (Observable{Any}(1), nothing)), Set{String}(), nothing, Asset[Asset("js", "knockout", "/home/henrikj/.julia/packages/Knockout/HReiN/src/../assets/knockout.js"), Asset("js", "knockout_punches", "/home/henrikj/.julia/packages/Knockout/HReiN/src/../assets/knockout_punches.js"), Asset("js", nothing, "/home/henrikj/.julia/packages/InteractBase/8TTmI/src/../assets/all.js"), Asset("css", nothing, "/home/henrikj/.julia/packages/InteractBase/8TTmI/src/../assets/style.css"), Asset("css", nothing, "/home/henrikj/.julia/packages/Interact/PENUy/src/../assets/bulma_confined.min.css")], Dict{Any, Any}("options_js" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"options_js\"]()) ? (this.valueFromJulia[\"options_js\"]=true, this.model[\"options_js\"](val)) : undefined})")], "index" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"index\"]()) ? (this.valueFromJulia[\"index\"]=true, this.model[\"index\"](val)) : undefined})")]), WebIO.ConnectionPool(Channel{Any}(32), Set{AbstractConnection}(), Base.GenericCondition{ReentrantLock}(Base.IntrusiveLinkedList{Task}(Task (runnable, started) @0x00007f14334644c0, Task (runnable, started) @0x00007f14334644c0), ReentrantLock(nothing, 0x00000000, 0x00, Base.GenericCondition{Base.Threads.SpinLock}(Base.IntrusiveLinkedList{Task}(nothing, nothing), Base.Threads.SpinLock(0)), (5000, 1, 0)))), WebIO.JSString[WebIO.JSString("function () {\n    var handler = (function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init: function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            });\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0)\n                     return;\n                stringified(str);\n            });\n            ko.applyBindingsToNode(\n                element,\n                {\n                    value: stringified,\n                    valueUpdate: allBindings

┌ Warning: Requested interp_dt conflicts with max_length
└ @ GuidingCenterOrbits ~/.julia/packages/GuidingCenterOrbits/upKwA/src/orbit.jl:456
┌ Warning: Requested interp_dt conflicts with max_length
└ @ GuidingCenterOrbits ~/.julia/packages/GuidingCenterOrbits/upKwA/src/orbit.jl:456
┌ Warning: Requested interp_dt conflicts with max_length
└ @ GuidingCenterOrbits ~/.julia/packages/GuidingCenterOrbits/upKwA/src/orbit.jl:456
┌ Warning: Requested interp_dt conflicts with max_length
└ @ GuidingCenterOrbits ~/.julia/packages/GuidingCenterOrbits/upKwA/src/orbit.jl:456
[ Info: Saved animation to /home/henrikj/Codes/OWCF/temp.gif
┌ Warning: Requested interp_dt conflicts with max_length
└ @ GuidingCenterOrbits ~/.julia/packages/GuidingCenterOrbits/upKwA/src/orbit.jl:456
┌ Warning: Requested interp_dt conflicts with max_length
└ @ GuidingCenterOrbits ~/.julia/packages/GuidingCenterOrbits/upKwA/src/orbit.jl:456


Movie and plots saved successfully! Please switch off 'save_plots'.


[ Info: Saved animation to /home/henrikj/Codes/OWCF/plt_anim_5.0_0.5_3.45.gif
[ Info: Saved animation to /home/henrikj/Codes/OWCF/temp.gif
┌ Warning: Requested interp_dt conflicts with max_length
└ @ GuidingCenterOrbits ~/.julia/packages/GuidingCenterOrbits/upKwA/src/orbit.jl:456
┌ Warning: Requested interp_dt conflicts with max_length
└ @ GuidingCenterOrbits ~/.julia/packages/GuidingCenterOrbits/upKwA/src/orbit.jl:456
[ Info: Saved animation to /home/henrikj/Codes/OWCF/temp.gif
┌ Warning: Requested interp_dt conflicts with max_length
└ @ GuidingCenterOrbits ~/.julia/packages/GuidingCenterOrbits/upKwA/src/orbit.jl:456
